In [169]:
from langchain.agents import Tool, initialize_agent, AgentType, load_tools
from dotenv import load_dotenv
from langchain.utilities import (
    WikipediaAPIWrapper,
    GoogleSearchAPIWrapper,
)
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain import OpenAI, Wikipedia

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)
from langchain.chat_models import ChatOpenAI

In [170]:
load_dotenv()

True

In [171]:
myTopic = "Tourism in Egypt"

## Getting Keywords


In [172]:
wiki = WikipediaAPIWrapper()
google = GoogleSearchAPIWrapper()
duck = DuckDuckGoSearchRun()

In [173]:
llm_keywords = OpenAI(temperature=0.2)

In [174]:
# keyword_extractor_tools = load_tools(['wikipedia'],llm=llm_keywords)
keyword_extractor_tools = [
    Tool(
        name="Google Search",
        description="Useful when you want to get information from Google about single topic.",
        func=google.run,
    ),
    Tool(
        name="DuckDuckGo Search Evaluation",
        description="Useful to evaluate the results of Google Search and add any missing information about specific topic.",
        func=duck.run,
    ),
]

In [175]:
keyword_agent = initialize_agent(
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_name="Keyword extractor",
    agent_description="You are a helpful AI that helps the user to get the important keyword list from the search results about specific topic.",
    llm=llm_keywords,
    tools=keyword_extractor_tools,
    verbose=True,
)

In [176]:
keyword_agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nGoogle Search: Useful when you want to get information from Google about single topic.\nDuckDuckGo Search Evaluation: Useful to evaluate the results of Google Search and add any missing information about specific topic.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Google Search, DuckDuckGo Search Evaluation]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [177]:
keyword_list = keyword_agent.run(
    f"Search about {myTopic} and use the results to get the important keywords related to {myTopic} to help to write a blog about {myTopic}."
)



> Entering new  chain...


 I need to find out what people are searching for when they look for information about Tourism in Egypt.
Action: Google Search
Action Input: "Tourism in Egypt"
Observation: In 2020, Egypt's tourism revenue dropped by about 70% to US$4 billion, reducing tourist arrivals to 3.5 million from 13.1 million in 2019. As stated by the ... Egypt, a country linking northeast Africa with the Middle East, dates to the time of the pharaohs. Egypt Tourism: Tripadvisor has 2360234 reviews of Egypt Hotels, Attractions, and Restaurants making it your best Egypt resource. Tourism in Egypt. Egypt recorded a total of 13 million tourists in 2019, ranking 19th in the world in absolute terms. The fact that larger countries ... Apr 24, 2021 ... Tourism in Egypt has been buffeted by political instability and terrorism. But the pandemic has dealt the industry its biggest blow in ... Terrorists may attack with little or no warning, and have targeted diplomatic facilities, tourist locations, transportation hubs, 

In [178]:
print(keyword_list)

Keywords related to Tourism in Egypt include: tourism revenue, pharaohs, Egypt Tourism, Tripadvisor, political instability, terrorism, diplomatic facilities, tourist locations, transportation hubs, markets/shopping malls, ancient sites, Red Sea coast, GDP, non-commodity exports, workforce, Travel Awaits, hotels, Red Sea, and footfall.


## Getting Title and SubTitle


In [179]:
title_llm = OpenAI()  # temperature=0.7

title_tools = [
    Tool(
        name="Intermediate Answer",
        description="Useful for when you need to ask with search about single topic.",
        func=google.run,
    ),
]

self_ask_with_search = initialize_agent(
    title_tools, title_llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)
title = self_ask_with_search.run(
    f"Suggest a titel for a blog about {myTopic}?",
)
subtitle = self_ask_with_search.run(
    f"Suggest a suitable subtitle for a blog about {myTopic}?",
)
print("title", title)
print("subtitle", subtitle)



> Entering new  chain...
 No.
So the final answer is: Exploring the Wonders of Egypt: A Tourist's Guide

> Finished chain.


> Entering new  chain...
 No.
So the final answer is: Exploring the Wonders of Egypt

> Finished chain.
title Exploring the Wonders of Egypt: A Tourist's Guide
subtitle Exploring the Wonders of Egypt


## Get search results


In [180]:
google_results = google.run(myTopic)
wiki_results = wiki.run(myTopic)
duck_results = duck.run(myTopic)
wiki_query_results = WikipediaQueryRun(api_wrapper=wiki).run(myTopic)

## Summarise the search results


In [181]:
# summarize the results separately
prompt = """Please Provide a summary of the following essay
The essay is: {essay}.
The summary is:"""
prompt_template = PromptTemplate(
    template=prompt,
    input_variables=["essay"],
)
summarize_llm = OpenAI(temperature=0)

In [182]:
summary_chain = LLMChain(
    llm=summarize_llm,
    prompt=prompt_template,
)
google_summary = summary_chain.run(essay=google_results)
wiki_summary = summary_chain.run(essay=wiki_results)
duck_summary = summary_chain.run(essay=duck_results)
wiki_query_summary = summary_chain.run(essay=wiki_query_results)

In [183]:
text_spitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n", "\t", "\r", "\f", "\v", "\0", "\x0b", "\x0c"],
    chunk_size=1000,
    chunk_overlap=500,
)

In [184]:
docs = text_spitter.create_documents(
    [google_results, wiki_results, duck_results, wiki_query_results]
)
num_docs = len(docs)
summary_chain2 = load_summarize_chain(
    llm=summarize_llm,
    chain_type="map_reduce",
)

In [185]:
tot_summary = summary_chain2.run(docs)

In [186]:
summary_tools = [
    Tool(
        name="Wikipedia",
        func=wiki.run,
        description="Search engine useful when you want to get information from Wikipedia about single topic.",
    ),
    Tool(
        name="Intermediate Answer",
        func=WikipediaQueryRun(api_wrapper=wiki).run,
        description="Search engine useful for when you need to ask with search",
    ),
    Tool(
        name="Google Search",
        description="Search engine useful when you want to get information from Google about single topic.",
        func=google.run,
    ),
    Tool(
        name="DuckDuckGo Search",
        description="Search engine useful when you want to get information from DuckDuckGo about single topic.",
        func=duck.run,
    ),
]

In [187]:
summary_agent = initialize_agent(
    summary_tools,
    llm=summarize_llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_name="Summary Agent",
    verbose=True,
)

In [188]:
tot_summary2 = summary_agent.run(
    f"can you provide me a summary about {myTopic} from each search engine separately? \ then use this information to combine all the summaries together to get a blog about {myTopic}."
)



> Entering new  chain...
 I need to get a summary from each search engine and then combine them together.
Action: Wikipedia
Action Input: Tourism in Egypt
Observation: Page: Tourism in Egypt
Summary: Tourism is one of the leading sources of income, crucial to Egypt's economy. At its peak in 2010, the sector employed about 12% of workforce of Egypt, serving approximately 14.7 million visitors to Egypt, and providing revenues of nearly $12.5 billion  as well as contributing more than 11% of GDP and 14.4% of foreign currency revenues.

Page: Terrorism and tourism in Egypt
Summary: Terrorism and tourism in Egypt is when terrorist attacks are specifically aimed at Egypt's tourists. These attacks often end in fatalities and injuries and has an immediate and sometimes lasting effect on the industry. Attacks take many forms; blowing up an airplane carrying tourists, drive-by shootings of tourists, knife attacks on tourists and suicide bombings in a location where tourists are congregated. On

## Create a chain to write the first draft of the blog


In [249]:
prompt_writer = """You are an expert online blogger with expert writing skills and I want you to only write out the breakdown of each section of the blog on the topic of {topic} 
using the following information:
keywords: {keywords}.
The title is: {title}.
The subtitle is: {subtitle}.
google results: {google_results}.
wiki results: {wiki_results}.
duck results: {duck_results}.
wiki query results: {wiki_query_results}.
google summary: {google_summary}.
wiki summary: {wiki_summary}.
duck summary: {duck_summary}.
wiki query summary: {wiki_query_summary}.
The results summary is: {summary}.
The final blog will be {wordCount} words long, with an introduction at the beginning and a conclusion at the end of the blog.
use the following template to write the blog:
[BLOG]
[title]
{title}
[subtitle]
{subtitle}
[introduction]
[body]
[conclusion]
[END BLOG]
"""

prompt_writer_template = PromptTemplate(
    template=prompt_writer,
    input_variables=[
        "topic",
        "title",
        "subtitle",
        "google_results",
        "wiki_results",
        "duck_results",
        "wiki_query_results",
        "google_summary",
        "wiki_summary",
        "duck_summary",
        "wiki_query_summary",
        "summary",
        "keywords",
        "wordCount",
    ],
)
writer_llm = OpenAI(temperature=0)

In [250]:
writer_chain = LLMChain(
    llm=writer_llm,
    prompt=prompt_writer_template,
    verbose=True,
)


In [251]:
draft1 = writer_chain.run(
    topic=myTopic,
    title=title,
    subtitle=subtitle,
    google_results=google_results,
    wiki_results=wiki_results,
    duck_results=duck_results,
    wiki_query_results=wiki_query_results,
    google_summary=google_summary,
    wiki_summary=wiki_summary,
    duck_summary=duck_summary,
    wiki_query_summary=wiki_query_summary,
    summary=tot_summary + tot_summary2,
    keywords=keyword_list,
    wordCount=2000,
)



> Entering new  chain...
Prompt after formatting:
You are an expert online blogger with expert writing skills and I want you to only write out the breakdown of each section of the blog on the topic of Tourism in Egypt 
using the following information:
keywords: Keywords related to Tourism in Egypt include: tourism revenue, pharaohs, Egypt Tourism, Tripadvisor, political instability, terrorism, diplomatic facilities, tourist locations, transportation hubs, markets/shopping malls, ancient sites, Red Sea coast, GDP, non-commodity exports, workforce, Travel Awaits, hotels, Red Sea, and footfall..
The title is: Exploring the Wonders of Egypt: A Tourist's Guide.
The subtitle is: Exploring the Wonders of Egypt.
google results: In 2020, Egypt's tourism revenue dropped by about 70% to US$4 billion, reducing tourist arrivals to 3.5 million from 13.1 million in 2019. As stated by the ... Egypt, a country linking northeast Africa with the Middle East, dates to the time of the pharaohs. Egypt Tou

In [252]:
print(draft1)


Introduction
Egypt is a country steeped in history and culture, with a vibrant population and a thriving tourism industry. From the ancient pyramids of Giza to the bustling markets of Cairo, Egypt is a destination that offers something for everyone. In this blog, we will explore the wonders of Egypt, from its tourism revenue to its cultural attractions, and provide a comprehensive guide for tourists looking to experience the best of what the country has to offer. 

Body
Tourism Revenue: Tourism is a major source of income for Egypt, contributing 11% of GDP in 2010. However, the industry has faced challenges due to political instability and terrorism, resulting in a decrease in tourism and a loss of tourism dollars to other countries. In 2020, Egypt's tourism revenue dropped by 70% to US$4 billion due to the pandemic, reducing tourist arrivals to 3.5 million from 13.1 million in 2019. Despite this, Egypt has seen a significant increase in tourism revenue, with revenue rising 71% to $4.

In [253]:
print("len draft1", len(draft1))

len draft1 1265


## Evaluate the first draft